In [1]:
import pandas as pd
import torch
from transformers import BertTokenizerFast, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTE
from datasets import Dataset
from torch.nn import CrossEntropyLoss
from transformers import EarlyStoppingCallback
import numpy as np

In [2]:
train_df = pd.read_csv('/kaggle/input/mlprojectdataset/train.csv')
test_df = pd.read_csv('/kaggle/input/mlprojectdataset/test.csv')

with open("/kaggle/input/mlprojectdataset/clinical-stopwords.txt", 'r') as f:
    stopwords = set(f.read().splitlines())

In [3]:
def remove_stopwords(text, stopwords):
    return ' '.join([word for word in text.split() if word.lower() not in stopwords])

train_df['description'] = train_df['description'].apply(lambda x: remove_stopwords(x, stopwords))
test_df['description'] = test_df['description'].apply(lambda x: remove_stopwords(x, stopwords))

label_encoder = LabelEncoder()
train_df['label'] = label_encoder.fit_transform(train_df['disease'])
test_df['label'] = label_encoder.transform(test_df['disease'])

In [4]:
tokenizer = BertTokenizerFast.from_pretrained('bert-base-cased', clean_up_tokenization_spaces=False)

def tokenize_data(examples):
    return tokenizer(examples['description'], padding='max_length', truncation=True, max_length=256)

train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

train_dataset = train_dataset.map(tokenize_data, batched=True)
test_dataset = test_dataset.map(tokenize_data, batched=True)

train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Map:   0%|          | 0/853 [00:00<?, ? examples/s]

Map:   0%|          | 0/212 [00:00<?, ? examples/s]

In [5]:
X_smote = train_df['description']
y_smote = train_df['label']

smote = SMOTE(random_state=42)
_, y_resampled = smote.fit_resample(np.zeros((len(X_smote), 1)), y_smote)  

resampled_indices = []
for class_label in np.unique(y_resampled):
    class_indices = np.where(y_smote == class_label)[0]
    class_resampled_indices = np.random.choice(class_indices, size=(y_resampled == class_label).sum(), replace=True)
    resampled_indices.extend(class_resampled_indices)


train_resampled_dataset = train_dataset.select(resampled_indices)

In [6]:
model = BertForSequenceClassification.from_pretrained('bert-base-cased', num_labels=len(label_encoder.classes_))

class_counts = train_df['label'].value_counts().sort_index()
class_weights = torch.tensor([1.0 / count for count in class_counts])
class_weights = class_weights.to(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
training_args = TrainingArguments(
    output_dir='./results',
    eval_strategy="epoch",         
    save_strategy="epoch",               
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=15,                  
    learning_rate=2e-5,                  
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    load_best_model_at_end=True,         
    metric_for_best_model="accuracy",
)


In [8]:
import torch
from sklearn.metrics import accuracy_score

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    logits_tensor = torch.tensor(logits)
    predictions = torch.argmax(logits_tensor, dim=-1)
    accuracy = accuracy_score(labels, predictions)
    return {"accuracy": accuracy}
    
from transformers import Trainer

class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        loss_fct = CrossEntropyLoss(weight=class_weights)
        loss = loss_fct(logits, labels)
        return (loss, outputs) if return_outputs else loss

In [9]:
trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=train_resampled_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],

In [10]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning, message=".*torch.cuda.amp.autocast.*")

In [11]:
with torch.amp.autocast('cuda', enabled=True):
    trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss,Accuracy
1,3.022900,2.816677,0.235849
2,2.520100,2.361238,0.518868
3,1.906100,1.789422,0.707547
4,1.395700,1.389440,0.834906
5,1.025800,1.113104,0.834906
6,0.825400,0.880651,0.877358
7,0.585900,0.717021,0.891509
8,0.445700,0.606941,0.910377
9,0.324200,0.535006,0.905660
10,0.241600,0.460231,0.924528


In [12]:
import torch
from sklearn.metrics import classification_report, accuracy_score

predictions = trainer.predict(test_dataset)

predictions_tensor = torch.tensor(predictions.predictions)
pred_labels = torch.argmax(predictions_tensor, axis=1)


print(classification_report(test_df['label'], pred_labels, target_names=label_encoder.classes_, zero_division=0))


test_accuracy = accuracy_score(test_df['label'], pred_labels)
print(f"Test Accuracy: {test_accuracy}")


                                 precision    recall  f1-score   support

                        allergy       0.90      0.90      0.90        10
                      arthritis       1.00      1.00      1.00        10
               bronchial asthma       0.91      1.00      0.95        10
           cervical spondylosis       0.83      1.00      0.91        10
                    chicken pox       1.00      0.80      0.89        10
                    common cold       0.91      1.00      0.95        10
                         dengue       1.00      0.80      0.89        10
                       diabetes       0.78      0.70      0.74        10
                  drug reaction       0.86      0.75      0.80         8
               fungal infection       1.00      1.00      1.00         9
gastroesophageal reflux disease       0.90      0.90      0.90        10
                   hypertension       0.91      1.00      0.95        10
                       impetigo       1.00      1.

In [13]:
from transformers import BertForSequenceClassification, BertTokenizerFast
import torch

model_save_path = './model'
tokenizer_save_path = './tokenizer'

model.save_pretrained(model_save_path)

tokenizer.save_pretrained(tokenizer_save_path)

print("Model and tokenizer saved!")

Model and tokenizer saved!


In [14]:
import joblib

label_encoder_save_path = './label_encoder.pkl'

joblib.dump(label_encoder, label_encoder_save_path)

print("Label encoder saved!")

Label encoder saved!
